Task 1
1. ownself explore the job and user dataset
2. Merge all the job skills related column and user want to work with column
3. Learn BERT and BI LSTM

In [1]:
## import libraries and datasets
import pandas as pd
df_user = pd.read_csv('Data/User/survey_results_public.csv')
df_job = pd.read_csv('Data/job_descriptions.csv')

Basic_extract = ['Age', 'Employment']
Edu_Work_extract = ['EdLevel', 'LearnCode', 'LearnCodeOnline', 'LearnCodeCoursesCert', 'YearsCode', 'DevType', 'Currency', 'CompTotal']
Tech_extract = [
    'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
    'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
    'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
    'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
    'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
    'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
    'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith',
    'OpSysPersonal use', 'OpSysProfessional use',
    'OfficeStackAsyncHaveWorkedWith', 'OfficeStackAsyncWantToWorkWith',
    'OfficeStackSyncHaveWorkedWith', 'OfficeStackSyncWantToWorkWith',
    'AISearchHaveWorkedWith', 'AISearchWantToWorkWith',
    'AIDevHaveWorkedWith', 'AIDevWantToWorkWith'
]


all_columns_to_extract = Basic_extract + Edu_Work_extract + Tech_extract
extracted_df_user = df_user[all_columns_to_extract]

def concatenate_tech_columns(row):
    # Filter out None or NaN values before joining
    filtered_values = [str(value) for value in row if pd.notnull(value)]
    return ' '.join(filtered_values)

# Apply the function across the Tech_extract columns and create a new column for the merged data
extracted_df_user['Tech_Features_Merged'] = extracted_df_user[Tech_extract].apply(concatenate_tech_columns, axis=1)

# Drop all columns in Tech_extract
extracted_df_user.drop(columns=Tech_extract, inplace=True)

/var/folders/5h/8hlpwpv526s12wp_5825s8kc0000gn/T/ipykernel_8290/3394226554.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_df_user['Tech_Features_Merged'] = extracted_df_user[Tech_extract].apply(concatenate_tech_columns, axis=1)
/var/folders/5h/8hlpwpv526s12wp_5825s8kc0000gn/T/ipykernel_8290/3394226554.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_df_user.drop(columns=Tech_extract, inplace=True)


In [2]:
print(extracted_df_user['Tech_Features_Merged'].head())



0                                                     
1    HTML/CSS;JavaScript;Python Bash/Shell (all she...
2    Bash/Shell (all shells);Go Haskell;OCaml;Rust ...
3    Bash/Shell (all shells);HTML/CSS;JavaScript;PH...
4    HTML/CSS;JavaScript;TypeScript HTML/CSS;JavaSc...
Name: Tech_Features_Merged, dtype: object


In [3]:
df_job['skills'] = df_job['skills'].str.lower().str.split(', ')
extracted_df_user['Tech_Features_Set'] = extracted_df_user['Tech_Features_Merged'].str.lower().str.split(' ').apply(set)
df_job['skills_set'] = df_job['skills'].apply(set)

/var/folders/5h/8hlpwpv526s12wp_5825s8kc0000gn/T/ipykernel_8290/990395516.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extracted_df_user['Tech_Features_Set'] = extracted_df_user['Tech_Features_Merged'].str.lower().str.split(' ').apply(set)


In [ ]:

print(df_job['skills_set'].head())
print(extracted_df_user['Tech_Features_Set'].head())
# Step 1: Add a dummy key to both DataFrames for the cross join
extracted_df_user['key'] = 1
df_job['key'] = 1

# Step 2: Perform the cross join
combined_df = pd.merge(extracted_df_user[['key', 'Tech_Features_Set']], df_job[['key', 'skills_set']], on='key').drop('key', axis=1)

# Step 3: Combine the sets into one column (optional, depending on your exact needs)
# If you wish to keep them separate for analysis, you can skip this step,
# as 'combined_df' already contains both sets.
combined_df['Combined_Skills'] = combined_df.apply(lambda row: row['Tech_Features_Set'].union(row['skills_set']), axis=1)

# Now, 'combined_df' has a 'Combined_Skills' column with the union of user tech features and job skills.

